# What is a Graph?


## Undirected Graphs

A *graph* $G$ consists of a nonempty set, $V(G)$, called the *vertices* of $G$, and a set $E(G)$ called the *edges* of $G$. An element of $V(G)$ is called a *vertex*. A vertex is also called a *node*; the words "vertex" and "node" are used interchangeably. An element of $E(G)$ is an *undirected* edge or simply an "edge". An undirected edge has two vertices $u\neq v$ called its endpoints. Such an edge can be represented by the two element set $\{u, v\}$. The notation $\langle u—v \rangle$ denotes this edge.
Both $\langle u—v \rangle$ and $\langle v—u \rangle$ define the same undirected edge, whose endpoints are $u$ and $v$.

![](./images/graph_example.png)

For example, let $H$ be the graph pictured in Figure above. The vertices of $H$
correspond to the nine dots, that is, $V(H) = \{a,b,c,d,e,f,g,h,i\}$

The edges correspond to the eight lines, that is,

$E(H) = \big\{\langle a—b \rangle,\langle a—c \rangle,\langle b—d \rangle,\langle c—d \rangle,\langle c—e \rangle,\langle e—f \rangle,\langle e—g \rangle,\langle h—i \rangle\big\} $


## Directed Graphs

A *directed graph* -or *digraph*- $G$ consists of a nonempty set $V(G)$, called the vertices of $G$, and a set $E(G)$, called the edges of $G$. An element of $V(G)$ is called a *vertex*. A vertex is also called a *node*; the words "vertex" and "node" are used interchangeably. An element of $E(G)$ is called a *directed edge*. A directed edge is also called an "arrow" or simply an "edge". A directed edge starts at some vertex $u$ called the *tail* of the edge, and ends at some vertex $v$ called the *head* of the edge.

![](./images/digraph_example.png)


### Vertex Degrees

The *in-degree* of a vertex in a digraph is the number of arrows coming into it, and similarly its *out-degree* is the number of arrows out of it. More precisely,

If $G$ is a *digraph* and $v \in V(G)$, then 
    
  * $indeg(v) ::= \big\vert e \in \{E(G)\, \vert\, head(e) = v \}\big\vert$
  * $outdeg(v) ::= \big\vert e \in \{E(G)\, \vert\, tail(e) = v \}\big\vert$


##### References:
The definitions above and the illustrations are taken from the book:
*Mathematics for Computer Science*, Eric Lehman, F. Tom Leighton, Albert R. Meyer
https://courses.csail.mit.edu/6.042/spring17/mcs.pdf

# Name Generator

Lets generate a directed graph -a social network-, which expresses endorsements between various people similar as in *LinkedIn*. The nodes of our digraph will be persons and the edges will represent endorsments. 


For the nodes, we will generate a set of "random" people. That is, we generate names as given in the USA randomly. A list of US American names can be received here:
http://www.census.gov/topics/population/genealogy/data/1990_census/1990_census_namefiles.html


In the following we download the files and generate three lists, one for female names, one for male names, and one for surnames.

In [1]:
import webget
import random


surnames_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.all.last'
female_names_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.female.first'
male_names_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.male.first'

webget.download(surnames_txt, to='./last_names.txt')
webget.download(female_names_txt, to='./female_names.txt')
webget.download(male_names_txt, to='./male_names.txt')


def get_names_from(file):
    names = []
    with open(file) as f:
        for line in f:
            name = line.split(' ')[0].title()
            names.append(name)
            
    return names

names = get_names_from('./last_names.txt')
female_names = get_names_from('./female_names.txt')
male_names = get_names_from('./male_names.txt')

print(len(names), len(female_names), len(male_names))

88799 4275 1219


In [2]:
%%bash
ls -ltr *names.txt

-rw-r--r--  1 thomasthimothee  staff  3107965 Mar  7 15:31 last_names.txt
-rw-r--r--  1 thomasthimothee  staff   149625 Mar  7 15:31 female_names.txt
-rw-r--r--  1 thomasthimothee  staff    42665 Mar  7 15:31 male_names.txt


We decide to let names be a two-element tuple consisting of a name and a surname.

In [2]:
def random_combination(list_a, list_b):
    a = random.choice(list_a)
    b = random.choice(list_b)
    
    return (a, b)


def generate_random_names(amount=1, gender='female'):
    pre_names = (female_names, male_names)
    
    if gender == 'female':
        pre_names = pre_names[0]
    elif gender == 'male':
        pre_names = pre_names[1]
    elif gender == 'random':
        pre_names = random.choice(pre_names)

    random_names = []
    for idx in range(0, amount):    
        random_name = random_combination(pre_names, names)
        random_names.append(random_name)
        
    return random_names
    
generate_random_names(amount=10, gender='random')

[('Dusty', 'Hasberry'),
 ('Isreal', 'Instasi'),
 ('Mark', 'Vanelderen'),
 ('Josiah', 'Sabastian'),
 ('Dewitt', 'Galavis'),
 ('Jaime', 'Cirilo'),
 ('Shirley', 'Keyworth'),
 ('Otto', 'Bagshaw'),
 ('Gayle', 'Ardon'),
 ('Lupe', 'Kaczka')]

# Generating Nodes

Now, that we have our name generator in place we are good to start generating our *endorsment graph*.

For our example graph, we decide that it contains 100 persons, i.e., 100 nodes. Half of the nodes represents women and the other half represents men.

In [5]:
total_no_nodes = 100
no_nodes_per_sex = total_no_nodes // 2

female_names_list = generate_random_names(amount=no_nodes_per_sex, gender='female')
male_names_list = generate_random_names(amount=no_nodes_per_sex, gender='male')

all_names_list = female_names_list + male_names_list
person_list = all_names_list

print(len(all_names_list))
print(all_names_list[:20])

100
[('Celesta', 'Vergamini'), ('Myrle', 'Amburgey'), ('Chantel', 'Scalese'), ('Bernita', 'Politis'), ('Roxane', 'Hannibal'), ('Lashunda', 'Gilly'), ('Leonila', 'Pinneo'), ('Jeanice', 'Sagehorn'), ('Hassie', 'Carrig'), ('Lizbeth', 'Marcott'), ('Rebekah', 'Bookard'), ('Mercy', 'Frieson'), ('Taryn', 'Salon'), ('Dodie', 'Beaumonte'), ('Deonna', 'Scholz'), ('Tamesha', 'Cogill'), ('Tawny', 'Toalson'), ('Pandora', 'Sero'), ('Tenesha', 'Kaczorowski'), ('Azalee', 'Poncedeleon')]


# Generating Edges

endorsements randomly


Additionally, we say that it is not possible for a person to endorse herself.

In [6]:
%%bash
conda install -y tqdm

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.4.11

Please update conda by running

    $ conda update -n base conda




In [7]:
from tqdm import tqdm
from random import randint
from numpy.random import choice


def get_no_endorsments():
    relation_classes = {
        0: (0, 0),
        1: (1, 4),
        2: (5, 10),
        3: (11, 20),
        4: (21, 50),
        5: (51, 100)
    }
    p_distribution = [0.05, 0.36, 0.45, 0.1, 0.035, 0.005]

    no_relation_class = choice(list(relation_classes.keys()), 
                               p=p_distribution)
    no_endorsments = randint(relation_classes[no_relation_class][0], 
                             relation_classes[no_relation_class][1])
    return no_endorsments


def create_endorsements(node_list):
    """Creates a list of graph edges
    like [(1,10), (50,4), (99,20000), ...]"""
    endorsements = []
    idx_list = range(len(node_list))

    for idx, name_pair in tqdm(enumerate(node_list)):
        # get how many people this person (name_pair) endorses
        no_endorsments = get_no_endorsments()
        if no_endorsments > 0:
            # get a list of potential endorsements
            potential_endorsment_idxs = random.sample(idx_list, no_endorsments)
            if idx in potential_endorsment_idxs:
                # Removing myself in case I am in the list
                # cannot endorse myself
                idx_me_in_endorsements = potential_endorsment_idxs.index(idx)
                del potential_endorsment_idxs[idx_me_in_endorsements]
    
            endorsements += list(zip([idx for _ in range(len(potential_endorsment_idxs))], 
                                     potential_endorsment_idxs))
    return endorsements

endorsements = create_endorsements(person_list)

print(endorsements[:30])
print(person_list)

100it [00:00, 6491.32it/s]

[(0, 30), (0, 26), (1, 12), (1, 59), (1, 94), (1, 25), (1, 17), (1, 92), (1, 16), (1, 61), (1, 97), (1, 95), (1, 41), (1, 65), (1, 43), (1, 15), (1, 18), (1, 89), (1, 60), (1, 7), (1, 58), (2, 45), (2, 25), (2, 50), (2, 59), (2, 40), (3, 60), (4, 43), (4, 7), (4, 15)]
[('Celesta', 'Vergamini'), ('Myrle', 'Amburgey'), ('Chantel', 'Scalese'), ('Bernita', 'Politis'), ('Roxane', 'Hannibal'), ('Lashunda', 'Gilly'), ('Leonila', 'Pinneo'), ('Jeanice', 'Sagehorn'), ('Hassie', 'Carrig'), ('Lizbeth', 'Marcott'), ('Rebekah', 'Bookard'), ('Mercy', 'Frieson'), ('Taryn', 'Salon'), ('Dodie', 'Beaumonte'), ('Deonna', 'Scholz'), ('Tamesha', 'Cogill'), ('Tawny', 'Toalson'), ('Pandora', 'Sero'), ('Tenesha', 'Kaczorowski'), ('Azalee', 'Poncedeleon'), ('Toya', 'Rumberger'), ('Cecelia', 'Samide'), ('Madeline', 'Greider'), ('Sophia', 'Coury'), ('Amelia', 'Ching'), ('Belia', 'Vititow'), ('Therese', 'Ota'), ('Janean', 'Mikolajczak'), ('Kaye', 'Dennert'), ('Nevada', 'Kreft'), ('Collette', 'Pefanis'), ('Kiana', 

In [8]:
%%bash
conda install -y networkx=1.11
pip install pygraphviz

Solving environment: ...working... done

# All requested packages already installed.

  Using cached pygraphviz-1.3.1.zip
  Running setup.py bdist_wheel for pygraphviz: started
  Running setup.py bdist_wheel for pygraphviz: finished with status 'error'
  Complete output from command /Users/thomasthimothee/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/z1/txs736td4rqclbn48yv32vjh0000gp/T/pip-build-7zqthkt0/pygraphviz/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /var/folders/z1/txs736td4rqclbn48yv32vjh0000gp/T/tmpvirybeospip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.7-x86_64-3.6
  creating build/lib.macosx-10.7-x86_64-3.6/pygraphviz
  copying pygraphviz/__init__.py -> build/lib.macosx-10.7-x86_64-3.6/pygraphviz
  copying pygraphviz/agraph.py -> build/lib



==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.4.11

Please update conda by running

    $ conda update -n base conda


  Failed building wheel for pygraphviz
Command "/Users/thomasthimothee/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/z1/txs736td4rqclbn48yv32vjh0000gp/T/pip-build-7zqthkt0/pygraphviz/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /var/folders/z1/txs736td4rqclbn48yv32vjh0000gp/T/pip-gt7i2irf-record/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/z1/txs736td4rqclbn48yv32vjh0000gp/T/pip-build-7zqthkt0/pygraphviz/


In [9]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [10]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout, write_dot


def create_graph():
    graph = nx.DiGraph()
    graph.clear()

    # add node by node, needed to add attributes...
    print(len(all_names_list))

    for idx, name_pair in enumerate(all_names_list):
        graph.add_node(idx, name=' '.join(name_pair))
    
    # graph.add_nodes_from(all_names_list)
    graph.add_edges_from(endorsements)

    return graph


def draw_graph(graph):
    nx.draw(graph, pos=graphviz_layout(graph), 
            node_size=30, width=.05, cmap=plt.cm.Blues, 
            with_labels=True, node_color=range(len(graph)))
    
graph = create_graph()
draw_graph(graph)

#nx.write_gml(graph, './social_network.gml')

100


ImportError: ('requires pygraphviz ', 'http://pygraphviz.github.io/')

In [ ]:
print(graph.node[10]['name'])
print(graph.out_degree(10))
print(graph.out_edges(10))
print(graph.node[graph.out_edges(10)[0][1]])
print(graph.in_edges(10))

In [ ]:
print(graph.node)
print(graph.number_of_nodes())

## Who is the most interesting person in out network?

Likely, you are tempted to find the person in the graph, which has the highest in-degree. For example with code similar to the following.

In [12]:
import numpy as np


in_deg_vec = np.array([graph.in_degree(n) for n in graph.nodes()])
max_ind_deg = in_deg_vec.max()
print(np.argmax(in_deg_vec))
print(graph.node[np.argmax(in_deg_vec)]['name'])

15
Tamesha Cogill


However, there is an issue with this solution. To make yourself the most interesting person you would just have to create a lot of profiles of people endorsing you. This issue can be overcome by applying the *PageRank* algorithm.

# PageRank Algorithm



![from_book](images/pr_descr.png)

##### References:
The above description is from *Artificial Intelligence: A Modern Approach Third Edition* by Stuart J. Russell and Peter Norvig.

## Let's Implement PageRank Together!

Perhaps it is best, when we start implementing a function that encodes the *PageRank* formula.


$PR(p) = \frac{1-d}{N} + d \sum_{i}^{} \frac{PR(in_{i})}{C(in_{i})}$

In [13]:
big_n = graph.number_of_nodes() # or len(graph.nodes())
d = 0.85
global_pr = [1 for node in graph.nodes()]
global_pr = np.ones(graph.number_of_nodes())
print(global_pr)


def pr(node, graph):
    _sum = 1
    part_1 = (1 - 0.85) / graph.number_of_nodes()
    
    for _ in range(graph.in_edges(node)):
        _sum += pr()

    return part_1 + 1.85 * _sum





[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [20]:
#print(graph.node[27]['name'])

Carolin Ruppenthal


In [ ]:
def page_rank(node):
    damping = 0.85
    big_n = graph.number_of_nodes()
    pr_in = 
    
    pr_p = (1 - damping) / n + damping * 